<a href="https://colab.research.google.com/github/PraneethVasa/CHAT/blob/main/F1_Task_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Importing Required Libraries***

1. **pandas, numpy**: Used for data manipulation and numerical operations.
2. **datetime**: To handle date conversions and calculate recency.
3. **random**: To generate random scores for business logic.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import random

## ***SettingUp Constants***

Defines key parameters:

1.  alpha, beta: Weights for calculating the combined score (frequency and recency).
2. purchasing_power: Represents the consultant’s budget.
3. k: Maximum bundle size.
4. C_max: Maximum cost allowed for a bundle.
5. gamma, delta: Weights for scoring candidates.
6. n_categories: Maximum categories per bundle.
7. num_bundles: Number of bundles to generate per consultant.

In [ ]:
alpha = 0.4
beta = 0.6
purchasing_power = 100.0
k = 7   # size of bundles
C_max = 100.0
theta = 1
gamma = 0.25
delta = 0.75
n_categories = 7  # max number of categories allowed per bundle
num_bundles = 4  # number of bundles

Reads a CSV file named '*sample_data_1000_C.csv*' into a DataFrame.

In [ ]:
df = pd.read_csv('/content/sample_data_1000_C.csv')
df.head()

,CODEBELISTA,CODPRODUCTOSAP,DESCATEGORIA,DESMARCA,PRECIOOFERTA,FECHAPROCESO
0,44109905,200112294,MAQUILLAJE,CYZONE,9.50,2023-02-09
1,36949902,200086399,FRAGANCIAS,LBEL,19.95,2023-02-02
2,49968221,200089498,TRATAMIENTO CORPORAL,ESIKA,36.90,2023-02-10
3,46114531,210102388,BIJOUTERIE,ESIKA,59.90,2023-02-21
4,50368645,210100620,COMPLEMENTOS,CYZONE,74.90,2023-03-01


## ***Data Pre-processing***

Data Cleaning
Column names are renamed for clarity.
Missing values in the category column are filled with "OTHERS."

Dates are converted to datetime format for aggregation.

In [ ]:
#Renaming the columns
column_rename_map={
                "CODEBELISTA": "consultant_id",
                "CODPRODUCTOSAP": "product_id",
                "DESCATEGORIA": "category",
                "DESMARCA": "brand",
                "PRECIOOFERTA": "price",
                "FECHAPROCESO": "date"
            }
df.rename(columns=column_rename_map, inplace=True)
df.head()

,consultant_id,product_id,category,brand,price,date
0,44109905,200112294,MAQUILLAJE,CYZONE,9.50,2023-02-09
1,36949902,200086399,FRAGANCIAS,LBEL,19.95,2023-02-02
2,49968221,200089498,TRATAMIENTO CORPORAL,ESIKA,36.90,2023-02-10
3,46114531,210102388,BIJOUTERIE,ESIKA,59.90,2023-02-21
4,50368645,210100620,COMPLEMENTOS,CYZONE,74.90,2023-03-01


In [ ]:
# Loop over each column and print unique values
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'consultant_id': [44109905 36949902 49968221 46114531 50368645  3441296 50086135 49621434
 46587790 40764801 45494314 33777957 45697525 34191115 49588232 50476650
 50349691 45869830 50837122 50843157 48346014 49599102 50639266 50437760
 49642350 48433154 49330626 48341519 50224287 48765629 46518039 50015882
 49961014 49434669 44242133 44479885 48823106 44196425 11976964 33090625
 41317310 39625989 11692060 49864310 50034054 46669258 49794401 42759880
 49728603 47430550 47948835 49086741 50147800  4623541 43767909 44661489
 46627121 49931492 49705387 44047470 49975686 44045680 49506570 33905173
 50052273 49657315 31826446 49566263 46219902 49397003 46611063 30185862
 49470762 49864469 44413795 33529260 47499070 50014223 48212689 42498750
 48297765 47097371 49740433 49885083 49931050 49075782 42382701 49794657
 45680231 34978506 47880688 10459362 42265901 49377649 49561199 49814240
 39630249 49206860 49389949 50051617 50172864 50291766 46779789 48855490
 48674135 

In [ ]:
#Filling Null Values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36310 entries, 0 to 36309
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   consultant_id  36310 non-null  int64  
 1   product_id     36310 non-null  int64  
 2   category       36228 non-null  object 
 3   brand          36310 non-null  object 
 4   price          36310 non-null  float64
 5   date           36310 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.7+ MB


In [ ]:
df['category'].fillna('OTHERS', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36310 entries, 0 to 36309
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   consultant_id  36310 non-null  int64  
 1   product_id     36310 non-null  int64  
 2   category       36310 non-null  object 
 3   brand          36310 non-null  object 
 4   price          36310 non-null  float64
 5   date           36310 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.7+ MB


<ipython-input-7-d823663f0706>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['category'].fillna('OTHERS', inplace=True)


In [ ]:
# Convert `date` column to datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36310 entries, 0 to 36309
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   consultant_id  36310 non-null  int64         
 1   product_id     36310 non-null  int64         
 2   category       36310 non-null  object        
 3   brand          36310 non-null  object        
 4   price          36310 non-null  float64       
 5   date           36310 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.7+ MB


**Daily Aggregation**
Groups data by consultant_id and date to compute frequency and daily_total_spent

In [ ]:
# Perform daily aggregation
group_by_columns = ["consultant_id", "date"]
daily_agg = df.groupby(group_by_columns, as_index=False).agg({
    "product_id": "count",
    "price": "sum"
})

# Rename columns for clarity
daily_agg.rename(columns={
    "product_id": "frequency",
    "price": "daily_total_spent"
}, inplace=True)

daily_agg.head()

,consultant_id,date,frequency,daily_total_spent
0,3441296,2022-12-27,10,263.40
1,3441296,2023-02-07,8,273.30
2,3441296,2023-02-15,1,23.90
3,3441296,2023-02-28,7,261.45
4,3441296,2023-04-11,17,391.10


**IQR and Purchasing Power** Interquartile Range (IQR) is calculated for daily_total_spent to identify spending patterns.
average_purchasing_power is set to Q3 (75th percentile of spending).

In [ ]:
# Calculate IQR for daily_total_spent
Q1 = np.percentile(daily_agg["daily_total_spent"], 25)
Q3 = np.percentile(daily_agg["daily_total_spent"], 75)
IQR = Q3 - Q1

# Add average purchasing power (Q3) as a new column
daily_agg["average_purchasing_power"] = Q3


**Merge Aggregates**
1. Daily aggregates are merged back into the main dataset.
2. Recency is calculated as days since the last purchase.

In [ ]:
# Merge the daily aggregates back into the original DataFrame
df = pd.merge(df, daily_agg, on=group_by_columns, how="left")
df.head()

,consultant_id,product_id,category,brand,price,date,frequency,daily_total_spent,average_purchasing_power
0,44109905,200112294,MAQUILLAJE,CYZONE,9.50,2023-02-09,17,427.40,430.195
1,36949902,200086399,FRAGANCIAS,LBEL,19.95,2023-02-02,16,403.90,430.195
2,49968221,200089498,TRATAMIENTO CORPORAL,ESIKA,36.90,2023-02-10,21,406.30,430.195
3,46114531,210102388,BIJOUTERIE,ESIKA,59.90,2023-02-21,11,469.10,430.195
4,50368645,210100620,COMPLEMENTOS,CYZONE,74.90,2023-03-01,5,202.55,430.195


In [ ]:
# Calculate recency in days
current_date = datetime.now()
df["recency"] = (current_date - df["date"]).dt.days
df.head()

,consultant_id,product_id,category,brand,price,date,frequency,daily_total_spent,average_purchasing_power,recency
0,44109905,200112294,MAQUILLAJE,CYZONE,9.50,2023-02-09,17,427.40,430.195,694
1,36949902,200086399,FRAGANCIAS,LBEL,19.95,2023-02-02,16,403.90,430.195,701
2,49968221,200089498,TRATAMIENTO CORPORAL,ESIKA,36.90,2023-02-10,21,406.30,430.195,693
3,46114531,210102388,BIJOUTERIE,ESIKA,59.90,2023-02-21,11,469.10,430.195,682
4,50368645,210100620,COMPLEMENTOS,CYZONE,74.90,2023-03-01,5,202.55,430.195,674


**Normalization Frequency and Recency Normalization**:
Converts raw values to a range between 0 and 1.
Handles edge cases where all values are identical.

Recency is calculated as days since the last purchase.

In [ ]:
# Normalize frequency
freq_min = df["frequency"].min()
freq_max = df["frequency"].max()

if freq_min != freq_max:
    df["frequency_normalized"] = (df["frequency"] - freq_min) / (freq_max - freq_min)
else:
    df["frequency_normalized"] = 0.5


In [ ]:
# Normalize recency
rec_min = df["recency"].min()
rec_max = df["recency"].max()
ddf?
if rec_min != rec_max:
    df["recency_normalized"] = (df["recency"] - rec_min) / (rec_max - rec_min)
else:
    df["recency_normalized"] = 0.5

Object `ddf` not found.


**Consultant-Level Metrics total_spent**: Total spending by each consultant.

**purchase_frequency**: Total number of purchases.

**unique_products**: Number of unique products purchased.

In [ ]:
# Consultant-level metrics
df["total_spent"] = df.groupby("consultant_id")["price"].transform("sum")
df["purchase_frequency"] = df.groupby("consultant_id")["product_id"].transform("count")
df["unique_products"] = df.groupby("consultant_id")["product_id"].transform("nunique")

In [ ]:
df.head()

,consultant_id,product_id,category,brand,price,date,frequency,daily_total_spent,average_purchasing_power,recency,frequency_normalized,recency_normalized,total_spent,purchase_frequency,unique_products
0,44109905,200112294,MAQUILLAJE,CYZONE,9.50,2023-02-09,17,427.40,430.195,694,0.280702,0.919946,4548.33,159,125
1,36949902,200086399,FRAGANCIAS,LBEL,19.95,2023-02-02,16,403.90,430.195,701,0.263158,0.929444,3459.60,116,102
2,49968221,200089498,TRATAMIENTO CORPORAL,ESIKA,36.90,2023-02-10,21,406.30,430.195,693,0.350877,0.918589,3674.05,137,120
3,46114531,210102388,BIJOUTERIE,ESIKA,59.90,2023-02-21,11,469.10,430.195,682,0.175439,0.903664,6815.04,161,143
4,50368645,210100620,COMPLEMENTOS,CYZONE,74.90,2023-03-01,5,202.55,430.195,674,0.070175,0.892809,9109.95,482,280


In [ ]:
df.to_csv("preprocessed_data_1000_C.csv", index=False)

## ***Combined Score for each row based on normalized frequency and recency***

**Combined Score** :
A weighted combination of normalized frequency and recency

**combined_score** = alpha * normalized_f + beta * normalized_r

In [ ]:
df = pd.read_csv('/content/preprocessed_data_1000_C.csv')

def normalize(value, min_value, max_value):
    if min_value == max_value:
        return 0
    return (value - min_value) / (max_value - min_value)

min_f = df['purchase_frequency'].min()
max_f = df['purchase_frequency'].max()
min_r = df['recency'].min()
max_r = df['recency'].max()

df['normalized_f'] = df['purchase_frequency'].apply(lambda x: normalize(x, min_f, max_f))
df['normalized_r'] = df['recency'].apply(lambda x: normalize(x, min_r, max_r))
df['combined_score'] = (alpha * df['normalized_f']) + (beta * df['normalized_r'])
df.head()

,consultant_id,product_id,category,brand,price,date,frequency,daily_total_spent,average_purchasing_power,recency,frequency_normalized,recency_normalized,total_spent,purchase_frequency,unique_products,normalized_f,normalized_r,combined_score
0,44109905,200112294,MAQUILLAJE,CYZONE,9.50,2023-02-09,17,427.40,430.195,694,0.280702,0.919946,4548.33,159,125,0.208995,0.919946,0.635565
1,36949902,200086399,FRAGANCIAS,LBEL,19.95,2023-02-02,16,403.90,430.195,701,0.263158,0.929444,3459.60,116,102,0.152116,0.929444,0.618513
2,49968221,200089498,TRATAMIENTO CORPORAL,ESIKA,36.90,2023-02-10,21,406.30,430.195,693,0.350877,0.918589,3674.05,137,120,0.179894,0.918589,0.623111
3,46114531,210102388,BIJOUTERIE,ESIKA,59.90,2023-02-21,11,469.10,430.195,682,0.175439,0.903664,6815.04,161,143,0.211640,0.903664,0.626854
4,50368645,210100620,COMPLEMENTOS,CYZONE,74.90,2023-03-01,5,202.55,430.195,674,0.070175,0.892809,9109.95,482,280,0.636243,0.892809,0.790183


**select_anchor_product(df)**:
Finds the product with the highest combined_score in the DataFrame, serving as the bundle's starting point.

**generate_candidates(bundle, df)**:
Returns products not already in the bundle by filtering out existing product IDs.

**category_score(product, bundle)**:
Checks if a product's category exists in the bundle, returning 1 for a match or 0 otherwise.

**business_score(product)**:
Generates a random score for a product as a placeholder for advanced business logic.

In [ ]:
def select_anchor_product(df):
        return df.loc[df['combined_score'].idxmax()]

def generate_candidates(bundle, df):
        bundle_product_ids = {p['product_id'] for p in bundle}
        return df[~df['product_id'].isin(bundle_product_ids)]

def category_score(product, bundle):
    return 1 if product['category'] in [b['category'] for b in bundle] else 0

def business_score(product):
    return random.random()

**score_candidates(candidates, bundle)**:
Computes a composite score for candidate products based on their category match with the bundle and business logic scores.

**build_bundle(df, anchor_product, purchasing_power, k, C_max, theta, n_categories)**:
Constructs a product bundle starting with the anchor product, adhering to constraints like max size, cost threshold, and category limits.

In [ ]:
def score_candidates(candidates, bundle):
        candidates = candidates.copy()
        candidates['category_score'] = candidates.apply(lambda p: category_score(p, bundle), axis=1)
        candidates['business_score'] = candidates.apply(business_score, axis=1)
        candidates['score'] = gamma * candidates['category_score'] + delta * candidates['business_score']
        return candidates

def build_bundle(df, anchor_product, purchasing_power, k, C_max, theta, n_categories):

        bundle = [anchor_product]
        current_total_cost = anchor_product['price']
        categories_in_bundle = {anchor_product['category']}

        # Exclude the anchor product from candidates
        candidates = df[df['product_id'] != anchor_product['product_id']]
        candidates = score_candidates(candidates, bundle)
        candidates = candidates[candidates['price'] <= theta * C_max].sort_values(by='score', ascending=False)

        for _, candidate in candidates.iterrows():
            if len(bundle) >= k:
                break
            if candidate['category'] in categories_in_bundle:
                # n_categories = 1 => do not add new categories
                if current_total_cost + candidate['price'] <= C_max:
                    bundle.append(candidate)
                    current_total_cost += candidate['price']
                    # We do not add a new category if n_categories=1
            else:
                # If your logic allows new category, incorporate that here
                pass

        return bundle

**Purpose:** This below snippet generates multiple product bundles for each consultant.

**Logic:**

1. Iterates through each consultant's products (consultant_id).
2. For each consultant, creates a specified number of bundles (num_bundles).
3. Selects an anchor product using select_anchor_product.
4. Constructs a bundle using build_bundle, considering constraints like size, cost, and category limits.
5. Assigns a unique bundle ID, consultant ID, and flags the anchor product (is_anchor).
5. Appends all products in the bundle to the bundles list with metadata.







In [ ]:
bundles = []
for consultant_id, group in df.groupby("consultant_id"):
            for bundle_index in range(num_bundles):
                anchor_product = select_anchor_product(group)
                bundle = build_bundle(
                    group,
                    anchor_product,
                    purchasing_power,  # from config
                    k,
                    C_max,
                    theta,
                    n_categories
                )
                unique_bundle_id = f"{consultant_id}_Bundle_{bundle_index+1}"
                for idx, product in enumerate(bundle):
                    product_copy = product.copy()
                    product_copy["consultant_id"] = consultant_id
                    product_copy["bundle_id"] = unique_bundle_id
                    product_copy["is_anchor"] = 1 if idx == 0 else 0
                    bundles.append(product_copy)

In [ ]:
bundles

[consultant_id                        3441296
 product_id                         200095159
 category                    CUIDADO PERSONAL
 brand                                  ESIKA
 price                                   7.38
 date                              2022-12-27
 frequency                                 10
 daily_total_spent                      263.4
 average_purchasing_power             430.195
 recency                                  738
 frequency_normalized                0.157895
 recency_normalized                  0.979647
 total_spent                          7206.76
 purchase_frequency                       261
 unique_products                          194
 normalized_f                        0.343915
 normalized_r                        0.979647
 combined_score                      0.725354
 bundle_id                   3441296_Bundle_1
 is_anchor                                  1
 Name: 1793, dtype: object,
 consultant_id                        3441296
 produ

In [ ]:
data = pd.DataFrame(bundles)
data.head()

,consultant_id,product_id,category,brand,price,date,frequency,daily_total_spent,average_purchasing_power,recency,...,purchase_frequency,unique_products,normalized_f,normalized_r,combined_score,bundle_id,is_anchor,category_score,business_score,score
1793,3441296,200095159,CUIDADO PERSONAL,ESIKA,7.38,2022-12-27,10,263.40,430.195,738,...,261,194,0.343915,0.979647,0.725354,3441296_Bundle_1,1,NaN,NaN,NaN
18988,3441296,200078931,CUIDADO PERSONAL,ESIKA,13.27,2023-06-08,16,385.90,430.195,575,...,261,194,0.343915,0.758480,0.592654,3441296_Bundle_1,0,1.0,0.984574,0.988431
27276,3441296,200064341,CUIDADO PERSONAL,ESIKA,21.90,2022-12-27,10,263.40,430.195,738,...,261,194,0.343915,0.979647,0.725354,3441296_Bundle_1,0,1.0,0.890042,0.917532
3149,3441296,200108807,CUIDADO PERSONAL,ESIKA,7.38,2024-07-03,16,354.53,430.195,184,...,261,194,0.343915,0.227951,0.274337,3441296_Bundle_1,0,1.0,0.884674,0.913505
22714,3441296,200095465,CUIDADO PERSONAL,ESIKA,11.90,2023-06-08,16,385.90,430.195,575,...,261,194,0.343915,0.758480,0.592654,3441296_Bundle_1,0,1.0,0.861828,0.896371


Save the DataFrame to a CSV file named ***bundels_data_1000_C.csv***.

In [ ]:
data.to_csv('bundels_data_1000_C.csv', index=False)

In [ ]:
# Loop over each column and print unique values
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")